# Laboratory 02 - FFT and Spectrum Analysis 

## MAE 3120, Spring 2020

## Grading Rubric

Procedures, Results, Plots, Tables - 60%

Discussion Questions - 25%

Neatness - 15%

## Introduction and Background

In this lab we use Python and the NI-DAQmx API for digital data acquisition. Using the ***DAQ*** Jupyter Notebook developed for this class, instructions are issued to the data acquisition hardware, either inside the PC or external to the PC (the hardware we use in our lab is connected through the USB port). The ***DAQ*** can be configured to record data to files, change sampling parameters, and display a live output of your sampled signal. 

The goal of this tutorial is to provide you with your first experience using the ***DAQ*** notebook to perform data acquisition. You will use the ***DAQ*** to take samples and plot voltage data and to illustrate some limitations of digital data acquisition systems. 

To help verify that you have configured the ***DAQ*** properly before performing trials, you will learn how to use ***NI MAX*** (a software provided by National Instruments). 

In spectral analysis the goal is to determine the frequency content of a signal. Aliasing can be a serious problem with digital data acquisition if the experimenter is not careful. Significant measurement errors called ***aliasing*** errors are possible if the waveform is not sampled at high enough frequency. To avoid aliasing, the ***sampling rate*** must be at least twice the maximum frequency of the measured signal. This restriction is called the ***Nyquist criterion***. Signal aliasing occurs when waveforms are sampled at frequencies below the Nyquist frequency. Aliased signals appear to have frequencies (and possibly even waveform *shapes*) that differ from those of the actual signal. For adequate resolution of the waveform shape, data should be sampled at a much higher frequency – typically at least five times the Nyquist frequency, if possible.

Digital PC-based data acquisition will not totally replace oscilloscopes, at least not in the near future. The reason is sampling frequency. The maximum sampling frequency of modern PC A/D systems is typically less than a MHz (megahertz). By comparison, a good digital oscilloscope may sample as high as several GHz (gigahertz)!

The fast Fourier transform (FFT) is a computationally efficient form of the more general discrete Fourier transform (DFT), which is itself a discretized version of the even more general Fourier transform (FT). Like Fourier series analysis, FFT analysis enables us to calculate the frequency content of a signal. Fourier series analysis is useful for continuous, periodic, analog signals of known fundamental frequency. FFT analysis, on the other hand, is useful for discretely sampled (digital) data, and can be applied even if the signal is not periodic. With FFT analysis, the fundamental frequency of a periodic signal does not have to be known a priori. NumPy has built-in FFT features, which are utilized in this lab. 

For $N$ sampled data points at sampling frequency $f_s$, the most useful output of an FFT calculation is the frequency spectrum or amplitude spectrum, which is a plot of modified FFT amplitude versus frequency. The frequency spectrum shows the relative importance or contribution of discrete frequencies, which range from zero to $f_s\,/\,2$. (The factor of two is a direct result of the Nyquist criterion.) The number of discrete frequencies on the frequency spectrum plot is $N\,/\,2 + 1$. This is half of the number of discretely sampled data points in the original signal, plus one extra since we typically plot both extreme values – from zero Hz (DC component) to the folding frequency $f_\textit{folding}$. 

Here are some useful definitions for FFTs:

- $N$ is the ***total number of discrete data points*** taken. $N$ is an input parameter, chosen by the user.<br><p></p>

- $f_s$ is the ***sampling frequency***, in Hz. $f_s$ is an input parameter, chosen by the user. *All other properties of the FFT, including sampling time, maximum frequency, frequency resolution, etc., are determined solely from these two inputs, $N$ and $f_s$.*<br><p></p>

- $T$ is the ***total sampling time***, and is calculated as $T = N\,/\,f_s$. To increase the sampling time, we must either *increase* the number of data points, or *decrease* the sampling frequency (or both).<br><p></p>

- $f_\textit{folding}$ is the ***folding frequency***, also called $f_\textit{max}$, the ***maximum frequency***. $f_\textit{folding} = f_s\,/\,2$. $f_\textit{folding}$ is the maximum frequency plotted on the frequency spectrum plot, since $f_\textit{folding}$ is the maximum frequency at which reliable information about the signal can be calculated, due to the Nyquist criterion. The only way to increase $f_\textit{folding}$ is to increase the sampling frequency.<br><p></p>

- $\Delta f$ is the ***frequency resolution*** or ***frequency increment*** of the frequency spectrum. $\Delta f = 1\,/\,T = f_s\,/\,N$. On the frequency spectrum plot, amplitudes of the FFT are plotted at $N\,/\,2 + 1$ discrete frequencies, each separated by $\Delta f$. In other words, the discrete values of $f$ are $0$, $\Delta f$, $2 \Delta f$, $3 \Delta f$, ... , $[(N\,/\,2\,– 1)] \Delta f$. (The amplitude at exactly $f_\textit{folding}$, i.e., at $(N\,/\,2) \Delta f$, is also plotted; this results in a total of $(N\,/\,2) + 1$ discrete frequencies, counting both $f = 0$ and $f = f_\textit{folding}$). The *only* way to increase the frequency resolution is to increase sampling time.<br><p></p>

Here is a summary of some useful techniques and rules to remember when calculating FFTs:

- To get better frequency resolution for a fixed sampling frequency, increase the number of data points.<br><p></p>

- To get better frequency resolution for a fixed number of data points, decrease the sampling frequency. (But be careful here not to let $f_s$ fall below the Nyquist criterion limit).<br><p></p>

- To get frequency component information at higher frequencies, increase the sampling frequency.<br><p></p>

- To reduce ***leakage*** in the frequency spectrum, do one or more of the following:<br><p></p>

  - Increase the number of sampled data points $N$ (at the cost of more computer time).<br><p></p>
  
  - Decrease the sampling frequency $f_s$ (but do not sample at such a low frequency that the Nyquist criterion is violated).<br><p></p>
  
  - Multiply the time signal by a ***windowing*** function prior to taking the FFT (at the cost of throwing away a significant portion of the signal, in particular data points near the start and finish of the time trace).

## Objectives

- Practice data acquisition with digital data acquisition systems.<br><p></p>

- Construct simple first-order anti-aliasing (low-pass passive) filters.<br><p></p>

- Learn a simple way to sum two voltage signals.<br><p></p>

- Examine the usefulness of an anti-aliasing filter.

## Equipment

- Computer<br><p></p>

- Software: NI MAX, Jupyter<br><p></p>

- Hardware: National Instrument CompactDAQ cDAQ-9174, NI-9201 C Series Voltage Input Module <br><p></p>

- Function/waveform generator, along with appropriate cables<br><p></p>

- Oscilloscope<br><p></p>

- Resistors (value to be determined by you)<br><p></p>

- 2 capacitors of nominal value .1 μF (.1 × 10<sup>-6</sup> F)<br><p></p>

- Breadboard (plugged in for the ground connection, but turned off – power not needed for this lab)<br><p></p>

- Various BNC and banana cords and breadboard jumper wires as needed

## Procedure

### Part I - Introduction to **NI MAX** and Hardware Configuration

In this section, you will use the ***NI MAX*** software to verify that you have properly setup your hardware. The CompactDAQ Data Acquisition System (cDAQ) reads from four input modules. In this lab, we will be using the *NI 9201* module. 

1. Verify that ***NI MAX*** and appropriate drivers are installed on your computer (see installation instructions if using personal computer). <br><p></p>

- Ensure the *NI 9201* card is in the furthest left spot on the cDAQ. <br><p></p>

- Connect the cDAQ to the computer using the USB cable and plug it in to a power source.<br><p></p>

- ***NI MAX*** may automatically launch after the cDAQ is connected and powered. Open ***NI MAX*** manually if this does not occur. <br><p></p>

- Select the *Devices and Interfaces* tab under *My System* in ***NI MAX***. There, select the device with a name similar to `NI cDAQ-9174 "cDAQ1"`. <br><p></p>

- Select the first module (`1: NI 9201 "cDAQ1Mod1"`). If the *NI 9201* card is not the first module, change its location and then click the "Refresh" button at the top. Take note of the *Name* displayed on the right side of the screen (`cDAQ1Mod1`) as you may need this later when configuring the ***DAQ*** program. <br><p></p>

- Select *Test Panels* from the top right menu. <br><p></p>

- Set the Rate to *1000 Hz*, and the Mode to *Continuous*. <br><p></p>

- Click *Start* to observe a live graph of the analog input. <br><p></p>

- A low voltage signal should be observed around 0V. This is noise being measured by the device as no voltage source is connected to it. As long as there are no errors here, you can proceed with your data acquisition in the next part of the procedure.<br><p></p>

- Should you wish to view a live output of the signal from the module at any point, revisit the *Test Panels* screen in ***NI MAX***.

### Part II - Digital Data Acquisition

In this section, you will use the ***DAQ*** Jupyter Notebook to test limitations of digital data acquisition systems. 

1. Open the ***DAQ*** Jupyter Notebook located in the *Labs* folder. Read below for information on how to use the `acquire` function. <br><p></p>

- In the second cell, you will notice the `acquire` function. This function will be used throughout the course to acquire data using Python. <br><p></p>

- The docstring provides a simplified explanation of how to use certain variables in the function. A more detailed explanation is provided below. <br><p></p>

- `acquire` requires two variables to be passed, `N` and `fs`; all other arguments are optional and can be used as needed. To use an optional argument, include the arguments name and the value you wish to assign to it (e.g. `max_channel=7`). <br><p></p>

- Arguments

 - `N (int)`: Number of samples to measure. <br><p></p>

 - `fs (float)`: Sampling frequency in Hz (Hertz, cycles per second). <br><p></p>

 - `max_channel (int)`: Maximum voltage channel to read from. If you look at the side of the *NI 9201* cards, you will notice that it lists channels from `ai0` to `ai7`. Setting this to `0` will result in data only being read from the `ai0` channel. Setting this to `7` will result in data being read from channels `ai0`-`ai7`. The default value is `0`. <br><p></p>

 - `file_out (String)`: Output directory and file name of initial output. This is the name and location of the output file. The default value is `''`, which results in no file output. An example of a valid file name would be `'C:\\Users\\Josh\\Downloads\\Lab2_0.csv'`. All file names **must** end in an underscore followed by a one or two digit number, then the CSV file extension (e.g. `_0.csv`). Do not forget that Python requires the double backslash (`\\`) in Strings.<br><p></p>

 - `output (String)`: Type of output. `'X'` is the default value and outputs no file. `'N'` creates a new file every trial. `'R'` replaces data from the last trial. `'A'` appends new data to the last trial's file. If no file exists initially, a file will be created so long as `'X'` is not selected. <br><p></p>

 - `time_sep (float)`: Time increment to update live plot in seconds. By default, the live plot updates every `1` second. It cannot update at any faster rate. This value does not need to be changed unless you are acquiring data for a long period of time and/or at a high sampling rate (> 15 seconds). If you are acquiring data for more than 1 minute, this value should be changed to a number around `30`. <br><p></p>

 - `zero_bound (bool)`: Determines whether live graph is bounded to 0. By default, this value is `True`, meaning that you will still be able to see the data recorded at 0 seconds while the live plot is updating. If you want to see the data from only the current `time_sep` value, set this to `False`. <br><p></p>

 - `daq_name (String)`: Name of DAQ card/system. By default, this is `'cDAQ1Mod1'`. This is the String from *Part I - Step 6*. While you should not need to change this value, if the card you are using is in a different slot, the name may be different. <br><p></p>
 
 - `mod_type (String)`: Type of DAQ card. By default, this is `'ai'` for an analog input. `'tc'` is used for a thermocouple. `'pr'` is used for an analog input with small differential voltage. <br><p></p>
 
 - `plt_sp_fft (bool)`: Plots frequency spectrum of data using Fast Fourier Transform (FFT). The default value is `False`. This argument only works with one channel input. <br><p></p>
 
 - `han_window (bool)`: Applies a Hanning window to the plot output. This does not change the saved data. The default value is `False`. <br><p></p>

- Example Usage

 - 100 samples of data, sampling rate of 1000 Hz, no file output: `acquire(100, 1000)`<br><p></p>

 - 100 samples of data, sampling rate of 1000 Hz, create a new file after every trial, file location location is 'C:\Users\Josh\Downloads': `acquire(100, 1000, file_out='C:\\Users\\Josh\\Downloads\\Lab2_0.csv', output='N')`<br><p></p>

 - 300e3 samples of data, sampling rate of 1000 Hz, no file output, refresh the plot every 30 seconds, view only the last 30 seconds of data: `acquire(300e3, 100, time_sep=30, zero_bound=False)`<br><p></p>
 
6. While performing trials, make sure you save the measured data and can access the data after the lab section. You will need it to generate plots in your lab report. <br><p></p>

- Your lab report must include a table with the following columns: 

  - Run number / filename<br><p></p>
  
  - Lab section (which step in the procedures)<br><p></p>
  
  - Signal<br><p></p>
    
    - Frequency (Hz) and Waveform Type *(e.g. Sine 20)*<br><p></p>
  
    - Amplitude (Vpp)<br><p></p>
  
    - DC Offset (VDC)<br><p></p>
  
  - Acquisition<br><p></p>
  
    - Sampling Rate (Hz)<br><p></p>
  
    - Number of Samples<br><p></p>
    
  - Comments *(e.g. Curve looks continuous)*

### Part III - Leakage effect

1. Remember to be organized when acquiring data. Generate a table that includes all the runs and associated parameters you will acquire in this lab and report this matrix of experiments in your lab report.<br><p></p>

- Connect the output of the function generator to the oscilloscope and also to the first channel of the *NI 9201* module using BNC cables and a BNC T-adapter. <br><p></p>

- Turn on the function generator and oscilloscope. <br><p></p>

- Using the `acquire` function, set `plt_sp_fft` to `True`. This will display the frequency plot when a FFT is applied to your data. Additionally, set the `time_sep` to `120` to prevent the graph from updating.<br><p></p> 

- Set a 10 Hz sine wave, 0V DC Offset, and 2V peak-to-peak amplitude on the waveform generator. Keep monitoring your signal on the oscilloscope. For each of the steps below, save the time history signal to file. Use Python to recreate each spectrum. In your report you are expected to report both the time series and frequency spectra. <br><p></p>

- Set $f_s = 200\text{ Hz}$, $N = 256$.<br><p></p>

- To try to reduce the leakage, try first to increase the sampling rate. Set $f_s = 1000\text{ Hz}$, $N = 256$.<br><p></p>

- Try the following settings. Set $f_s = 25\text{ Hz}$, $N = 256$.<br><p></p>

- Try the following settings. Set $f_s = 25\text{ Hz}$, $N = 512$. What can you conclude about the spectral accuracy of our system?<br><p></p>

- Finally, try the following settings. Set $f_s = 25.6\text{ Hz}$, $N = 512$. This corresponds to a “perfect FFT”, can you think why?

### Part IV - Windowing of FFT

Keep the same parameters as above for the signal generator. Keep monitoring your signal on the oscilloscope.<br><p></p>

1. Using the `acquire` function, set `han_window` to `True`. This will apply a Hanning windowing function. It has the following formula and appearance:<br><p></p>

$$u_\textit{Hanning}(t) = \frac{1}{2} \left(1 - cos \frac{2 \pi t}{T}\right)$$

<img src="img/Hanning.png" width=480>

- Redo the measurements from *Part II* and save data for each condition. Do you observe any improvement?<br><p></p>

- Add a 1V DC Offset and redo the measurements. What do you observe?<br><p></p>

- By now you should know how to optimize the spectral response of a system. Now create triangular waves of similar frequency and select the proper sampling rate, period, and windowing. How many harmonics do you observe? 

### Part V - Aliasing

Here you will develop an anti-aliasing filter and verify its behavior. You will use the same circuit for *Part IV*, so do not modify your circuit after this section.<br><p></p>

1. Using the same configuration than previously ($f_s = 51.2\text{ Hz}$), now input a waveform of $f = 40\text{ Hz}$ and demonstrate aliasing. Estimate the apparent frequency analytically and compare it to the measured data. Take screenshots for your report.<br><p></p>

- You will now construct a first-order low-pass filter using a combination of resistors and capacitors. A circuit diagram for a simple first-order low-pass filter is provided below, the cutoff frequency is determined by: $f_\textit{cutoff} = \pi RC\,/\,2$. Set the cutoff frequency at the sampling frequency record this signal without aliasing. Select the value of the resistor accordingly. Note: In all these circuits, it is critical that the ground bus be connected to a physical ground, such as the black ground post of the powered breadboard with the breadboard plugged in (but turned off) to secure the ground. <br><p></p>

- Test your filter by connecting the function generator to the filter input and the filter output to the DAQ board. Start with a sine wave input of about 5 Hz and amplitude 1V. Increase the frequency of the sine wave, doubling it every time, until you start noticing that the output amplitude decreases – the low-pass filter is doing its job. Keep increasing the frequency until the filter output amplitude is 90% attenuated – in other words, G = 0.1; the output amplitude is about 10% of the input amplitude, or 0.1V. At what frequency does this occur?

<img src="img/FFT%20Circuit.png" width=140>

### Part VI - Signal Reconstruction

A signal contaminated with a high-frequency noise will be simulated and its proper acquisition with an anti-aliasing filter studied in this section. This requires using advanced function in the waveform generator to generate the sum of two signals. The carrier wave is a sine wave, 10 Hz, 5V. The noise is a sine wave at 3.1 kHz with amplitude 1V. <br><p></p>

1. Program the sum of the two sines in the waveform generator. In order to sum high frequency noise to carrier signal, go to modulate button on waveform function generator. Turn on modulate and choose sum under type, internal under source. Choose sine as shape of the noise and give the sum amplitude and frequency as it mentioned above.<br><p></p>

- Monitor that you have the proper signal on the oscilloscope. <br><p></p>

- Send the signal directly to the DAQ system (i.e. without going through the anti-aliasing filter that you have created).<br><p></p> 

- Sample at 500 Hz with 1,024 data points per scan. You should observe the low frequency signal nicely, but the high frequency signal should yield some aliasing.<br><p></p>

- Save the time trace and recreate the frequency spectrum for your lab report. Estimate the frequency of the two signals from the frequency spectrum plot. Calculate the frequency resolution of your DAQ system for this sampling frequency and comment on the resolution of your signal.<br><p></p>

- Redo *Steps 4 & 5* with a sampling frequency of:<br><p></p>
  - 1 kHz <br><p></p>
  
  - 5 kHz<br><p></p>
  
  - 10 kHz<br><p></p>
  
- Run the summed signal through the low-pass filter created previously. The low-pass filter is now acting as an anti-aliasing filter. Predict the amplitude of the noise and compare to the quantization error of your DAQ board.<br><p></p>

- Reacquire signals for each sampling frequency, i.e. $f_s$ = 0.5, 1, 5, and 10 kHz. Record these traces into your lab report as well. Take particular note of the high frequency noise. Has it been reduced? Has it been totally eliminated? Record your observations. 

# Discussion Questions

1. *Part I*, for each of the test cases, calculate the frequency resolution and the energy contained at the signal frequency (10 Hz). Comment on:

  A. The resolution of the sine wave.
  
  B. The width of the spike on the frequency spectrum and the energy contained at 10 Hz vs what you would expect. Explain how the width changes for each condition and what is the source of the observed phenomenon and how it can be corrected. <br><p></p>
  
2. *Part I* conclusions:

  A. What is the benefit and drawbacks of increasing the sampling frequency?
  
  B. What is the benefit and drawbacks of increasing the sampling period?
  
  C. What is a “perfect FFT”?<br><p></p>
  
3. What is the effect of the Hanning windowing on your signal? Does it totally eliminate leakage?<br><p></p>

4. What is the effect of windowing when there is a DC offset in addition of the sinusoidal signal? What can you conclude about the mean of a signal on which windowing can be applied? Propose a procedure to apply windowing when the signal has non-zero mean.<br><p></p>

5. For *Part IV*, did your anti-aliasing filter effectively remove the high frequency “noise”, while at the same time preserving the desired low frequency signal? Why or why not? Explain.<br><p></p>

6. In *Part IV*, which frequency was the optimum to record your signal? Hint: think about the frequency resolution.

# Appendices 

## Appendix A - NI cDAQ-9174

<img src="img/cDAQ-9174.png" width=240 align="left"><br><br><br><br><br><br><br><br>

[Online Manual](https://www.ni.com/documentation/en/compactdaq-chassis/latest/cdaq-9174/overview/)

[User Manual](https://www.ni.com/pdf/manuals/372838e.pdf)

[Specification Sheet](https://www.ni.com/pdf/manuals/374045a.pdf)

## Appendix B - NI 9201

<img src="img/NI-9201.png" width=150 align="left"><br><br><br><br><br><br><br><br>

[HTML Manual](https://www.ni.com/documentation/en/c-series-voltage-input-module/latest/9201/overview/)

[Datasheet](https://www.ni.com/pdf/manuals/373783a_02.pdf)

**Signal Level**: ± 10V

**Channels**: 8 Single-Ended

**Max Sample Rate (Single Channel)**: 800 kS/s

**Max Sample Rate (Scanning)**: 500 kS/s

**Simultaneous** No

**ADC Resolution**: 12-Bit

**Type of ADC**: Successive approximation register (SAR)

<img src="img/NI-9201%20Circuit.png" width=480 align="left"><br><br><br><br><br><br><br><br><br>

<img src="img/NI-9201%20Sample%20Rate.png" width=480 align="left"><br><br><br><br><br><br><br><br><br><br><br><br>

<img src="img/NI-9201%20Accuracy.png" width=480 align="left"><br><br><br><br><br><br>

<img src="img/NI-9201%20Stability.png" width=480 align="left">